In [1]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np 


In [2]:
## Load the pickle files (trained model , scaler/ onehotencoder/labelencoder)  and do the prediction 

model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file :
    label_encoder_gender=pickle.load(file)
    
with open('One_encoder_Geography.pkl','rb') as file :
    One_encoder_Geography=pickle.load(file)
    
with open('scaler.pkl','rb') as file :
    scaler=pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [7]:
## change it to dataframe to avoid problems 

input_data = pd.DataFrame([input_data])

In [8]:
## deal with categorical feartures 

input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
geo_encoder = One_encoder_Geography.transform(input_data[['Geography']])
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=One_encoder_Geography.get_feature_names_out(['Geography']))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
input_data = pd.concat([input_data.drop('Geography',axis=1),geo_encoder_df],axis=1)
input_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
### scaling the data 

input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [12]:
#### predict churn 

prediction = model.predict(input_data_scaled)

prediction 

1/1 [==============================] - 0s 242ms/step


array([[0.03879994]], dtype=float32)

In [13]:
prediction_proba = prediction[0][0]
prediction_proba

0.03879994

In [14]:
if prediction_proba > 0.5 : 
    print('the customer is likely to churn ')
if prediction_proba < 0.5 :
    print('the customer is not likely to churn ')

the customer is not likely to churn 
